In [14]:
import json
import gurobipy as gp
from gurobipy import GRB, quicksum
import networkx as nx

# Function to read JSON data
def read_json_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# Replace 'AL_county.json' with the actual file path
json_data = read_json_data('AL_county.json')

# Initialize the Gurobi model
model = gp.Model("Integrated_Redistricting")

# Constants
num_districts = 7

for i in range(num_districts):
    print("i:", i)

total_population = 5024279  # Total population
target_population = total_population / num_districts  # Target population per district
#max_population_deviation = 0.5  #deviation
#races = ['White', 'Black', 'Hispanic', 'Asian', 'Other']  # Main race categories

#print(json_data)

#print(json_data['nodes'])

Counties = []

i = 0
for node in json_data['nodes']:
    Counties.append(i)
    i += 1
   
   
#print("Adj:", json_data['adjacency'])

edge_list = []

i = 0
for element in json_data['adjacency']:
    for neighbor in element:        
        edge = (i, neighbor['id'])
        edge_list.append(edge)
    i += 1    

#print("edge_list:", edge_list)


graph = nx.from_edgelist(edge_list)

print("Nodes:", graph.nodes)

print("Edges:", graph.edges)
   
range(num_districts)    
#print("Counties:", Counties)

#print("Range 7: ", range(num_districts))

#G = Graph.from_json(json_data)

#print("Nodes:", G.nodes)

# Decision variables for each district and race
#variables = {(district, race): model.addVar(vtype=GRB.CONTINUOUS, name=f"dist{district}_{race}")
             #for district in range(1, num_districts + 1) for race in races}
   
   
# How to get the population of each county
#for county in Counties:
#    print("Population:", json_data['nodes'][county]['P0010001'])
   

X = model.addVars(Counties, range(num_districts), vtype=GRB.BINARY)

# Assignment
model.addConstrs(quicksum(X[c,d] for d in range(num_districts)) == 1 for c in Counties)

U = 1.005 * target_population

L = 0.995 * target_population

# Population Balance
model.addConstrs(quicksum(json_data['nodes'][c]['P0010001']*X[c,d] for c in Counties) <= U for d in range(num_districts))
model.addConstrs(quicksum(json_data['nodes'][c]['P0010001']*X[c,d] for c in Counties) >= L for d in range(num_districts))

#Objective function

objective = quicksum(quicksum(X[u,j]*X[v,j] for (u,v) in graph.edges) for j in range(num_districts))

# Set the objective
model.setObjective(objective, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Print results
for county in Counties: 
    for district in range(num_districts):
        if X[county, district].X > 0.5:
            print((county, district))

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
Nodes: [0, 28, 54, 26, 58, 22, 64, 1, 46, 53, 33, 62, 37, 2, 56, 39, 45, 43, 17, 3, 15, 38, 49, 4, 32, 24, 14, 35, 5, 7, 31, 55, 61, 18, 6, 10, 30, 29, 8, 63, 41, 21, 44, 9, 60, 11, 51, 65, 42, 12, 47, 16, 59, 13, 40, 25, 27, 66, 57, 19, 52, 20, 48, 23, 50, 36, 34]
Edges: [(0, 28), (0, 54), (0, 26), (0, 58), (0, 22), (0, 64), (28, 1), (28, 21), (28, 26), (28, 62), (28, 37), (28, 54), (54, 8), (54, 21), (54, 32), (54, 58), (26, 22), (26, 37), (26, 36), (26, 45), (58, 4), (58, 14), (58, 32), (58, 64), (22, 45), (22, 64), (22, 23), (22, 50), (64, 14), (64, 27), (64, 50), (1, 46), (1, 53), (1, 33), (1, 62), (1, 37), (46, 9), (46, 33), (46, 40), (46, 51), (46, 53), (53, 13), (53, 25), (53, 40), (53, 62), (33, 9), (33, 65), (33, 37), (33, 56), (33, 36), (33, 39), (62, 21), (62, 25), (37, 36), (2, 56), (2, 39), (2, 45), (2, 43), (2, 17), (56, 39), (56, 65), (39, 36), (39, 45), (45, 23), (45, 36), (45, 43), (43, 17), (43, 23), (43, 52), (17, 52), (3, 15), (3,

In [15]:
# Function to read JSON data
def read_json_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# Read the data
# Replace '/path/to/AL_county.json' with the actual file path
json_data = read_json_data('AL_county.json')

# Initialize the Gurobi model
model = gp.Model("Redistricting")

# Define the district variables
num_districts = 6

target_population = 5024279 / 7   #target population is the population we want in each district

population = 5024279

district1 = 1
district2 = 2
district3 = 3
district4 = 4
district5 = 5
district6 = 6
district7 = 7

max_population_deviation = 0.5

factor = 1.5 # Giving priority to communities of interest

# Define decision variables
variables = {area: model.addVar(vtype=GRB.BINARY, name=f"district_{7}") for i, area in enumerate(json_data)}

# Legal Compliance (Binary variable: 1 for compliance, 0 for non-compliance)
legal_compliance = model.addVar(vtype=GRB.BINARY, name="Legal_Compliance")

# Add constraints
# Constraint: Legal compliance should be met
model.addConstr(legal_compliance == 1)

# Define the list of political subdivisions
political_subdivisions = [district1, district2, district3, district4, district5, district6, district7]

for subdivision in political_subdivisions:
    # Constraint: Each political subdivision should be assigned to one district
    model.addConstr(quicksum(variables[area] for area in json_data) == 1)  # Use quicksum here

# Suppose you have a list of communities of interest, where each community is a list of district variables.
communities_of_interest = [district1, district2, district3, district4, district5, district6, district7]

for community in communities_of_interest:
    # Constraint: Each community of interest should be assigned to one district
    model.addConstr(quicksum(variables[area] for area in json_data) == 1)  # Use quicksum here
   
for i in range(num_districts):
    # Constraint: Population deviation should be minimized
    model.addConstr(target_population - population <= max_population_deviation)
    model.addConstr(population - target_population >= max_population_deviation)

# Define objective function
objective = sum(variables[area] * factor for area in json_data)

# Set the objective
model.setObjective(objective, GRB.MINIMIZE)

# Optimize the model
model.optimize()




Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 27 rows, 6 columns and 71 nonzeros
Model fingerprint: 0x02a9607f
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.5000000
Presolve removed 27 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+00, best bound 1.500000000000e+00, gap 0.0000%


In [ ]:
print